# भाग 3: प्रगत दूरस्थ कार्यवाही साधने (Advanced Remote Execution Tools)


आधीच्या विभागात आपण फेडरेटेड लर्निंगचा (Federated Learning) वापर करून बनावट मॉडेलला प्रशिक्षण दिले. आपण  आपल्या मॉडेलवर .send () आणि .get () वर करून, प्रशिक्षण डेटाच्या ठिकाणी पाठवून, अद्यतनित करुन आणि परत आणून हे केले. तथापि, उदाहरणाच्या शेवटी आपल्याला हे समजले की लोकांच्या गोपनीयतेचे रक्षण करण्यासाठी आपल्याला अजून थोडे पुढे जाणे आवश्यक आहे. म्हणजेच .get () कॉल करण्यापूर्वी आपल्याला क्रमांकाची (average gradient)सरासरी पाहिजे आहे. अशा प्रकारे, आपण कोणाचेही नेमका ग्रेडियंट (Gradient) कधीच पाहणार नाही (अशा प्रकारे त्यांच्या गोपनीयतेचे रक्षण करणे अधिक चांगले !!!)

परंतु हे करण्यासाठी आम्हाला आणखी काही माहिती आवश्यक आहेतः

- दुसर्‍या कामगारांना(worker) थेट टेन्सर (Tensor) पाठविण्यासाठी पॉईंटर (Pointer)वापरा

आणि याव्यतिरिक्त, आपण येथे असताना आपण आणखी काही प्रगत टेन्सर (Tensor) ऑपरेशन्सबद्दल जाणून घेणार आहोत जे या उदाहरणात आणि भविष्यात काही मदत करेल!

लेखक:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote)- GitHub: [@Noob-can-Compile](https://github.com/Noob-can-Compile)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

# विभाग 3.1 - पॉइंटर्स ते पॉईंटर्स (Pointers तो Pointers)

तुम्हाला माहिती आहेच, पॉइंटरटेन्सर वस्तू (PointerTensor Object) सामान्य टेन्सरप्रमाणेच (Tensor) वाटतात. खरं तर, ते इतके टेन्सरसारखे आहेत की आपल्या पॉईंटर्सकडे पॉईंटर्स देखील असू शकतात. हे पहा!

In [ ]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [ ]:
# this is a local tensor
x = torch.tensor([1,2,3,4])
x

In [ ]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

In [ ]:
# now we can SEND THE POINTER to alice!!!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

### काय झालं?

तर, मागील उदाहरणात, आपण `x` नावाचे टेंसर (Tensor) तयार केले आणि ते Bob ला पाठविले, आपल्या स्थानिक मशीनवर पॉईंटर (Pointer) तयार करून (`x_ptr`).

त्यानंतर, आम्ही `x_ptr.send(alice)` कॉल केला ज्याने Alice **ला पॉईंटर (Pointer) पाठविला**.

टीप, यामुळे डेटा हलला नाही! त्याऐवजी, त्याने डेटाकडे पॉईंटर (Pointer)हलविला !!

In [ ]:
# As you can see above, Bob still has the actual data (data is always stored in a LocalTensor type). 
bob._objects

In [ ]:
# Alice, on the other hand, has x_ptr!! (notice how it points at bob)
alice._objects

In [ ]:
# and we can use .get() to get x_ptr back from Alice

x_ptr = pointer_to_x_ptr.get()
x_ptr

In [ ]:
# and then we can use x_ptr to get x back from Bob!

x = x_ptr.get()
x

### पॉइंटर वर अंकगणित(Arithmetic) -> पॉइंटर -> डेटा ऑब्जेक्ट

आणि अगदी सामान्य पॉइंटर्स (Pointers) प्रमाणेच, आपण या टेन्सरमध्ये (Tensor) अनियंत्रित PyTorch ऑपरेशन्स करू शकतो.

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x = torch.tensor([1,2,3,4,5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
p2p2x.get().get()

In [ ]:
bob._objects

In [ ]:
alice._objects

# विभाग 3.2 - पॉइंटर चेन ऑपरेशन्स

म्हणून शेवटच्या विभागात जेव्हा जेव्हा आपण .send () किंवा .get () ऑपरेशन कॉल केले तेव्हा त्या ऑपरेशनला आपल्या स्थानिक मशीनवरील टेन्सरवर थेट कॉल केले. तथापि, आपल्याकडे पॉइंटर्सची (Pointers) साखळी असल्यास, कधीकधी तुम्हाला साखळीच्या शेवटच्या पॉईंटरवर .get () किंवा .send () सारख्या ऑपरेशन्सवर कॉल करायचा असतो (जसे की एका कामगारांकडून (Worker) थेट दुसर्‍या कामगाराकडे (Worker) डेटा पाठवणे). हे साध्य करण्यासाठी, आपण विशेषत: या गोपनीयता संरक्षित ऑपरेशनसाठी तयार केलेली कार्ये वापरू इच्छित आहात.

ते ऑपरेशन्स आहेतः

- `my_pointer2pointer.move(another_worker)`

In [ ]:
# x is now a pointer to a pointer to the data which lives on Bob's machine
x = torch.tensor([1,2,3,4,5]).send(bob)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x = x.move(alice)

In [ ]:
print('  bob:', bob._objects)
print('alice:',alice._objects)

In [ ]:
x

उत्तम! आता आपण विश्वसनीय अ‍ॅग्रीगेटरचा वापर करून रिमोट ** ग्रेडियंट सरासरी (gradient averaging)** करण्यासाठी साधनांनी सुसज्ज आहोत!

# अभिनंदन !!! - समुदायात सामील होण्याची वेळ आली!


हे नोटबुक ट्यूटोरियल पूर्ण केल्याबद्दल अभिनंदन! आपण याचा आनंद घेत असल्यास आणि एआय(AI) आणि एआय सप्लाय चेन  (डेटा) च्या विकेंद्रित(Decentralized) मालकीच्या गोपनीयतेच्या संरक्षणाच्या दिशेने चळवळीत सामील होऊ इच्छित असाल तर आपण हे खालील प्रकारे करू शकता!

### Pysyft ला Github वर Star करा!

आमच्या समुदायाला मदत करण्याचा सर्वात सोपा मार्ग म्हणजे फक्त गिटहब(GitHub) रेपो(Repo) तारांकित(Star) करणे! हे आम्ही तयार करीत असलेल्या छान साधनांविषयी जागरूकता वाढविण्यास मदत करते.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### आमच्या Slack मध्ये सामील व्हा!


नवीनतम प्रगतीवर अद्ययावत राहण्याचा उत्तम मार्ग म्हणजे आमच्या समुदायामध्ये सामील होणे! आपण [http://slack.openmined.org](http://slack.openmined.org) येथे फॉर्म भरुन तसे करू शकता.

### एका कोड प्रोजेक्टमध्ये सामील व्हा!

आमच्या समुदायामध्ये योगदानाचा उत्तम मार्ग म्हणजे कोड योगदानकर्ता बनणे! कोणत्याही वेळी आपण (PySyft GitHub Issues Page) वर  जाऊ शकता आणि "Project" साठी फिल्टर करू शकता. हे आपण कोणत्या प्रकल्पांमध्ये सामील होऊ शकता याबद्दल विहंगावलोकन देणारी सर्व उच्च स्तरीय तिकिटे दर्शवेल! आपण एखाद्या प्रकल्पात सामील होऊ इच्छित नसल्यास, परंतु आपण थोडं कोडिंग करू इच्छित असाल तर आपण "good first issues" म्हणून चिन्हांकित गिटहब(GitHub) अंक शोधून आणखी "one off" मिनी-प्रकल्प(mini project) शोधू शकता.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करा

आपल्याकडे आमच्या कोडेबेसमध्ये योगदान देण्यास वेळ नसल्यास, परंतु तरीही आपल्याला समर्थन द्यावयाचे असल्यास आपण आमच्या मुक्त संग्रहात बॅकर देखील होऊ शकता. सर्व देणगी आमच्या वेब होस्टिंग आणि हॅकॅथॉन आणि मेटअप्स सारख्या इतर सामुदायिक खर्चाकडे जातात!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)